### Imports

In [18]:
import numpy as np
from bond_pricing import annuity_rate

### No-Arbitrage Price - Spot Curve

The starting place in no‐arbitrage bond valuation is the zero‐coupon (or spot) yield curve and these rates are used to value coupon bonds.

No-arbitrage price for a 4-year, 4% annual coupon payment corporate bond.

Assume that the sequence of zero-coupon bond yields is `[3.5%, 3.8%, 4.1%, and 4.2%]`, an upward-sloping spot curve. (“Spot rate” is a commonly used synonym for zero‐coupon rate.)

Generalization of the bond pricing equation:
$$PV=\frac{PMT}{(1+\mathit{z}_{1})^{1}}+\frac{PMT}{(1+\mathit{z}_{2})^{2}}+ \cdots + \frac{PMT+PV}{(1+\mathit{z}_{N})^{N}}$$

PV is the no‐arbitrage value of the N‐period bond—the sum of the present values of the cash flows, each of which is discounted using the zero‐coupon rate that corresponds to the period ($\mathit{z}_{1}, \mathit{z}_{2}, \cdots, \mathit{z}_{N}$), PMT is the coupon payment per period, and FV is the principal (usually taken to be 100 so the price can be interpreted as the percentage of par value).

Given a particular bond price, the yield to maturity is the internal rate of return (IRR) on the cash flows. An IRR is the uniform discount rate such that the sum of the present values of the future cash flows discounted at that particular interest rate for each time period equals the price of the bond.

The correct way to think about a **yield to maturity is as a summary statistic** about the cash flows on the bond which does not presume a flat yield curve. It is a **present value weighted average** of the sequence of zero‐coupon rates with most of the weight on the last cash flow. 

In [21]:
spot_rates1 = np.array([0.035, 0.038, 0.041, 0.042])
coupon_rate = 0.04
fv = 100
years = 4
per = 1

def bond_price_spot_rate(coupon_rate, fv, years, per, spot_rates):
  coupon = fv * coupon_rate * per 
  cash_flows = np.full(years, coupon) 
  cash_flows[years-1] += fv
  period = np.arange(1, years+1)
  pv_cf = cash_flows / (1 + spot_rates/per)**(period*per)
  return np.sum(pv_cf)

pv = bond_price_spot_rate(coupon_rate=coupon_rate, fv=fv, years=years, per=per, spot_rates=spot_rates1)
print(f'{pv = :0.3f}')
ytm = annuity_rate(n_periods=years, instalment=fv*coupon_rate, pv=pv, terminal_payment=fv)
print(f'{ytm = : 0.5f}')

pv = 99.342
ytm =  0.04182


In [22]:
spot_rates1 = np.array([0.035, 0.038, 0.041, 0.042]) # upward sloping zero-coupon curve
spot_rates2 = np.array([0.0492, 0.0469, 0.043, 0.0416]) # downward sloping spot curve
spot_rates3 = np.array([0.0213, 0.0282, 0.0365, 0.0425])
spot_rates4 = np.array([0.04182, 0.04182, 0.04182, 0.04182])

rates = [spot_rates1, spot_rates2, spot_rates3, spot_rates4]

for spot_rate in rates:
  pv = bond_price_spot_rate(coupon_rate=coupon_rate, fv=fv, years=years, per=per, spot_rates=spot_rate)
  print(f'{pv = :0.3f}')

pv = 99.342
pv = 99.342
pv = 99.342
pv = 99.342


### Bond Prices and Yields

The yield to maturity ($\mathit{y}$) per period is the internal rate of return given the cash flows.

$$PV=\frac{PMT}{(1+\mathit{y})^{1}}+\frac{PMT}{(1+\mathit{y})^{2}}+ \cdots + \frac{PMT+PV}{(1+\mathit{y})^{N}}$$

$$PV=\frac{PMT}{y}\times\left[1 - \frac{1}{(1+y)^{N}}\right]+\frac{FV}{(1+y)^{N}}$$

$$\frac{PV-FV}{FV}=\frac{c-y}{y}\times\left[1-\frac{1}{(1+y)^{N}}\right]$$

Here *c* is the coupon rate per period, PMT/FV. This expression indicates the connection between the price of the bond vis‐à‐vis par value and the coupon rate vis‐à‐vis the yield to maturity. These are the well‐known (and well‐remembered) rules: (1) If the bond is priced at par value (*PV = FV*), the coupon rate and the yield to maturity are equal (*c = y*); (2) if the price is a discount below par value (*PV < FV*), the coupon rate is less than the yield (*c < y*); and (3) if the price is a premium above par value (*PV > FV*), the coupon rate is greater than the yield (*c > y*). These rules apply to a coupon payment date when *N* is an integer.

Alternative arrangement of the bond pricing equation:
$$PV=PMT\left(\frac{1-(1+y)^{-N}}{y}\right)+\frac{FV}{(1+y)^N}$$
$$FV_{n}=PMT\left[\frac{(1+y)^{N}-1}{y}\right]+TV$$
$$PMT=\left[\frac{PV - \frac{FV}{(1+y)^N}}{\frac{1-(1+y)^{-N}}{y}}\right]$$

In [ ]:
def pv(r, n, pmt, fv, beg=False):
  old_settings = np.seterr(invalid='ignore')
  pvPMT = np.where(r == 0, n, np.divide(1 - (1+r)**-n, r)) * pmt
  np.seterr(**old_settings)
  pvFV = fv / (1 + r)**n
  
  return np.where(beg, (pvPMT + pvFV) * (1 + r), pvPMT + pvFV)

def fv(r, n, pmt, terminal_payment, beg=False):
  old_settings = np.seterr(invalid='ignore')
  fvPMT = np.where(r == 0, n, np.divide((1+r)**n - 1, r)) * pmt
  np.seterr(**old_settings)
  TV = terminal_payment
  
  return np.where(beg, fvPMT * (1 + r) + TV, fvPMT + TV)

def pmt(r, n, pv, fv):
    return (pv - (fv/(1+r)**n)) / np.divide(1 - (1+r)**-n, r)

### Yield Statistics

- **Yield-to-Maturity** is also called the *Redemption Yield*.
- **Current Yield** also called the *running yield* and the *income yield*; it is the annual coupon payment divided by the price of the bond. 
  - $\text{Coupon Yield}=\frac{PMT}{PV}$